In [ ]:
df = pd.read_csv('/Users/abdihussein/Downloads/fake_job_postings.csv')
df.head()

In [ ]:
df = df[['description', 'fraudulent']].dropna()
X = df['description']
y = df['fraudulent']  # (0 = real, 1 = fake)

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, average_precision_score

pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=20000)),
    ('clf', LogisticRegression(class_weight='balanced', solver='saga', max_iter=2000))
])

param_grid = {
    'tfidf__max_features': [5000, 10000, 20000],
    'clf__C': [0.01, 0.1, 1.0, 10.0],
    'clf__penalty': ['l1', 'l2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
g = GridSearchCV(pipe, param_grid, cv=cv, scoring=make_scorer(average_precision_score, needs_proba=True), n_jobs=-1, verbose=1)
g.fit(X_train, y_train)

print("best:", g.best_params_)
y_pred = g.predict(X_test)

In [ ]:
# Train/test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
#convert text into numberical features using TF_IDF 

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [ ]:
#Training the Linear Regression model

log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train_tfidf, y_train)

y_pred = log_reg.predict(X_test_tfidf)


In [ ]:
#Evaluating the Model
print("Accuracy results:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

